In [ ]:
using Logging
logger = ConsoleLogger(stdout)
# debuglogger = ConsoleLogger(stderr, Logging.Debug)
global_logger(logger)
using JLD

using Random
rng = MersenneTwister(1234)
import Dates

In [ ]:
using Plots
pyplot()

In [ ]:
include("utils/misc.jl")
using .MiscUtils
include("particle_1D/plot_results_1D.jl")
using .PlotRes1D
include("particle_1D/CPMP_bilin_game_1D.jl")
using .CPMPBilinGame1D
include("particle_1D/game_theory_contdom_1D.jl")
using .GameTheoryContdom1DUtils
include("particle_1D/lyapunov_1D.jl")
using .Lyapunov1DUtils

import ForwardDiff
import LinearAlgebra as LA

In [ ]:
ts = Dates.now()
ts_fsfriendly = Dates.format(ts, "yyyy-mm-ddTHHMMSS") # filesystem-friendly string for ts
resultdir = mkpath("results/particleMNE_badcase__$(ts_fsfriendly)")

## logging
logfile = "$resultdir/log.txt"
touch(logfile)
# open(logfile, "a") do f
#     write(f, "BAD CASE\n")
# end

In [ ]:
my_trig_badcase(x, y) = sin(4*pi*x) + sin(4*pi*y) + 2*cos(2*pi*(x+y))
gfun = my_trig_badcase
contour(-.5:.01:.5, -.5:.01:.5, my_trig_badcase, fill=true, aspect_ratio=1)

#### Preliminary run: go to neighborhood of solution

In [ ]:
## algo parameters
T = 200 # 500
mx = 10 # nb particles
my = 11
eta0_wx = 4e-2 # initial stepsize
eta0_wy = 4e-2
eta0_x  = 1e-3
eta0_y  = 1e-3
init_pos = "grid_unif" # "iid_unif" or "grid_unif"
extrasteps = 2 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4

In [ ]:
open(logfile, "a") do f
    write(f, "=== PRELIMINARY RUN ===\n")
    write(f, "T=$T\n")
    write(f, "mx=$mx\n")
    write(f, "my=$my\n")
    write(f, "eta0_wx=$(eta0_wx)\n")
    write(f, "eta0_wy=$(eta0_wy)\n")
    write(f, "eta0_x=$(eta0_x)\n")
    write(f, "eta0_y=$(eta0_y)\n")
    write(f, "init_pos=$(init_pos)\n")
    write(f, "extrasteps=$(extrasteps)\n")
end

In [ ]:
## preliminary run
copies_wx, copies_x, copies_wy, copies_y = run_CPMP(gfun, T, mx, my, eta0_wx, eta0_x, eta0_wy, eta0_y, init_pos; 
    extrasteps=extrasteps,
    rng=rng)
avg_wx, avg_x, avg_wy, avg_y = avg_iter(copies_wx, copies_x, copies_wy, copies_y)
save("$resultdir/prelimrun_iterates.jld", "copies_wx", copies_wx, "copies_x", copies_x, "copies_wy", copies_wy, "copies_y", copies_y)

In [ ]:
# Check that last iterate (wx0, x0, wy0, y0) is close to optimal, as measured by NI error
wx0, x0, wy0, y0 = copies_wx[:,T+1], copies_x[:,T+1], copies_wy[:,T+1], copies_y[:,T+1]
nierr0 = glob_NI_err(gfun, wx0, x0, wy0, y0; deltax=deltax, deltay=deltay)
gval0 = payoff(gfun, wx0, x0, wy0, y0)
open(logfile, "a") do f
    write(f, "Got last iterate of the preliminary run (wx0, x0, wy0, y0) with NI error $(nierr0)\n")
    write(f, "Payoff at (wx0, x0, wy0, y0): gval0=$(gval0)\n")
end
nierr0, gval0

In [ ]:
## Visually check the first variations at last iterate
plt_firstvar_x, plt_firstvar_y = plot_firstvars(gfun, wx0, x0, wy0, y0, gval0,
    resultdir)
plt_firstvar_x

In [ ]:
plt_firstvar_y

#### Aggregate particles and check we are still in a neighborhood of solution

In [ ]:
### Get aggregated representation of (wx0, x0, wy0, y0)
wx00, x00, mxstar = aggregate_particles_1D(wx0, x0; thresh_pos=5e-2)
wy00, y00, mystar = aggregate_particles_1D(wy0, y0; thresh_pos=5e-2)
dxstar_min = minimum([torus_dist_1D(x00[I], x00[J]) for I=1:mxstar for J=1:I-1])
dystar_min = minimum([torus_dist_1D(y00[I], y00[J]) for I=1:mystar for J=1:I-1])
open(logfile, "a") do f
    write(f, "Aggregating last iterate of the preliminary run.\n")
    write(f, "mxstar, mystar, dxstar_min, dystar_min: $mxstar, $mystar, $dxstar_min, $dystar_min\n")
end
mxstar, mystar, dxstar_min, dystar_min

In [ ]:
save("$resultdir/prelimrun_lastiterate.jld", "wx0", wx0, "x0", x0, "wy0", wy0, "y0", y0)
save("$resultdir/prelimrun_lastiterate_aggregated.jld", "wx00", wx00, "x00", x00, "wy00", wy00, "y00", y00)

In [ ]:
nierr00 = glob_NI_err(gfun, wx00, x00, wy00, y00; deltax=deltax, deltay=deltay)
gval00 = payoff(gfun, wx00, x00, wy00, y00)
open(logfile, "a") do f
    write(f, "Aggregated last iterate (wx00, x00, wy00, y00) has NI error $(nierr00)\n")
    write(f, "Payoff at (wx00, x00, wy00, y00): gval00=$(gval00)\n")
end
nierr00, gval00

In [ ]:
plt_firstvar_x, plt_firstvar_y = plot_firstvars(gfun, wx00, x00, wy00, y00, gval0,
    resultdir)
plt_firstvar_x

In [ ]:
plt_firstvar_y

#### Run CP-MDA (using hot and exact-parametrized initialization)

In [ ]:
## algo parameters
mx = mxstar # nb particles
my = mystar
T = 10 * 1000000
eta0_wx = 0.1 * 2e-3 # initial stepsize -- even be more generous and reduce the step-size compared to what will be used for CP-MP
eta0_wy = 0.1 * 2e-3
eta0_x  = 0.1 * 1e-4
eta0_y  = 0.1 * 1e-4
# init_pos = "grid_unif"
extrasteps = 1 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4

In [ ]:
open(logfile, "a") do f
    write(f, "=== RUN ON HOT EXACT-PARAMETRIZED INITIALIZATION ===\n")
    write(f, "T=$T\n")
    write(f, "mx=$mx\n")
    write(f, "my=$my\n")
    write(f, "eta0_wx=$(eta0_wx)\n")
    write(f, "eta0_wy=$(eta0_wy)\n")
    write(f, "eta0_x=$(eta0_x)\n")
    write(f, "eta0_y=$(eta0_y)\n")
    write(f, "extrasteps=$(extrasteps)\n")
end

In [ ]:
## run CPMDA from hot exact-parametrized initialization (wx00, x00, wy00, y00)
copies_wx, copies_x, copies_wy, copies_y = run_CPMP(gfun, T, mxstar, mystar, eta0_wx, eta0_x, eta0_wy, eta0_y, nothing; 
    wx00=wx00, x00=x00, wy00=wy00, y00=y00,
    extrasteps=extrasteps,
    rng=rng)
avg_wx, avg_x, avg_wy, avg_y = avg_iter(copies_wx, copies_x, copies_wy, copies_y)
save("$resultdir/iterates.jld", "copies_wx", copies_wx, "copies_x", copies_x, "copies_wy", copies_wy, "copies_y", copies_y)

In [ ]:
wxFF, xFF, wyFF, yFF = copies_wx[:,T+1], copies_x[:,T+1], copies_wy[:,T+1], copies_y[:,T+1]
nierrFF = glob_NI_err(gfun, wxFF, xFF, wyFF, yFF; deltax=deltax, deltay=deltay)
gvalFF = payoff(gfun, wxFF, xFF, wyFF, yFF)
open(logfile, "a") do f
    write(f, "Got last iterate (wxFF, xFF, wyFF, yFF) with NI error $(nierrFF)\n")
    write(f, "Payoff at (wxFF, xFF, wyFF, yFF): gvalFF=$(gvalFF)\n")
end
nierrFF, gvalFF

#### Compute and plot optimality metrics

In [ ]:
## plotting parameters
deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4
evalNIevery    = Int(floor(T/500)) # can go very precise since mxstar=mystar=2 small
evallocNIevery = Int(floor(T/500))
evalVevery     = Int(floor(T/50))
plotiterevery  = Int(floor(T/50))

In [ ]:
### compute and plot (global) NI error of iterates
plt_NI, plt_NI_log, nierrs = plot_NI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evalNIevery;
    deltax=deltax, deltay=deltay, skip_avg=true)
save("$resultdir/nierrs__every$(evalNIevery).jld", "nierrs", nierrs)
plt_NI_log

In [ ]:
### compute and plot local NI error of iterates
plt_locNI, plt_locNI_log, locnierrs = plot_locNI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evallocNIevery)
plt_locNI_log

In [ ]:
## plot first variations
plt_firstvar_x, plt_firstvar_y = plot_firstvars(gfun, wxFF, xFF, wyFF, yFF, gvalFF,
    resultdir)
plt_firstvar_x

In [ ]:
plt_firstvar_y

In [ ]:
# ## plot lyapunov function, using last iterate as reference point (estimate of the MNE, exact-parametrized)
# wxstar, xstar = 
# wystar, ystar = 

# alpha_x = 3
# alpha_y = 3
# lambda_x = 3 # (must tweak manually! paper uses lambda_x = eta_x^{-1/6} for proofs)
# lambda_y = 3
# tau_x = 1e-2
# tau_y = 1e-2

# plt_Vxy, plt_Vxy_log, Vweis_xy, Vposs_xy, Vtots_xy = plot_V_xy(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
#     wxstar, xstar, wystar, ystar, 
#     eta0_wx, eta0_x, eta0_wy, eta0_y,
#     resultdir, logfile,
#     evalVevery;
#     alpha_x=alpha_x, lambda_x=lambda_x, tau_x=tau_x,
#     alpha_y=alpha_y, lambda_y=lambda_y, tau_y=tau_y)
# plt_Vxy_log

In [ ]:
# this_Tstart = 1000 # (the xlabel in the resulting plot will not be adjusted!)
# this_Tend = 2000
# this_evalVevery = 1
# plt_Vxy, plt_Vxy_log, Vweis_xy, Vposs_xy, Vtots_xy = plot_V_xy(gfun, this_Tend-this_Tstart, copies_wx[:,this_Tstart:this_Tend], copies_x[:,this_Tstart:this_Tend], copies_wy[:,this_Tstart:this_Tend], copies_y[:,this_Tstart:this_Tend],
#     avg_wx, avg_x, avg_wy, avg_y,
#     wxstar, xstar, wystar, ystar, 
#     eta0_wx, eta0_x, eta0_wy, eta0_y,
#     resultdir, logfile,
#     this_evalVevery;
#     alpha_x=alpha_x, lambda_x=lambda_x, tau_x=tau_x,
#     alpha_y=alpha_y, lambda_y=lambda_y, tau_y=tau_y)
# plt_Vxy_log

In [ ]:
# ### compute and plot weight and position and total lyapunov function of iterates
# plt_V_x, plt_V_x_log, Vweis_x, Vposs_x, Vtots_x = plot_V(gfun, T, copies_wx, copies_x, avg_wx, avg_x,
#     wxstar, xstar,
#     eta0_wx, eta0_x,
#     resultdir, logfile, "x",
#     evalVevery;
#     alpha=alpha_x, lambda=lambda_x, tau=tau_x)

In [ ]:
# plt_V_y, plt_V_y_log, Vweis_y, Vposs_y, Vtots_y = plot_V(gfun, T, copies_wy, copies_y, avg_wy, avg_y,
#     wystar, ystar,
#     eta0_wy, eta0_y,
#     resultdir, logfile, "y",
#     evalVevery;
#     alpha=alpha_y, lambda=lambda_y, tau=tau_y)

#### Run CP-MP (using hot and exact-parametrized initialization)

In [ ]:
## algo parameters
alpha = 1.
scaling = 2
mx = mxstar # nb particles
my = mystar
T = Int(floor( 1000000/alpha^2 ))
eta0_wx = alpha * 2e-3 # initial stepsize
eta0_wy = alpha * 2e-3
eta0_x  = alpha * 1e-4
eta0_y  = alpha * 1e-4
# init_pos = "grid_unif"
extrasteps = 2 # extrasteps=1: CP-MDA, extrasteps=2: CP-MP

deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4

In [ ]:
open(logfile, "a") do f
    write(f, "=== RUN ON HOT EXACT-PARAMETRIZED INITIALIZATION ===\n")
    write(f, "T=$T\n")
    write(f, "mx=$mx\n")
    write(f, "my=$my\n")
    write(f, "eta0_wx=$(eta0_wx)\n")
    write(f, "eta0_wy=$(eta0_wy)\n")
    write(f, "eta0_x=$(eta0_x)\n")
    write(f, "eta0_y=$(eta0_y)\n")
    write(f, "extrasteps=$(extrasteps)\n")
end

In [ ]:
## run CPMP from hot exact-parametrized initialization (wx00, x00, wy00, y00)
copies_wx, copies_x, copies_wy, copies_y = run_CPMP(gfun, T, mxstar, mystar, eta0_wx, eta0_x, eta0_wy, eta0_y, nothing; 
    wx00=wx00, x00=x00, wy00=wy00, y00=y00,
    extrasteps=extrasteps,
    true_prox=true,
    rng=rng)
avg_wx, avg_x, avg_wy, avg_y = avg_iter(copies_wx, copies_x, copies_wy, copies_y)
save("$resultdir/iterates.jld", "copies_wx", copies_wx, "copies_x", copies_x, "copies_wy", copies_wy, "copies_y", copies_y)

In [ ]:
wxMP, xMP, wyMP, yMP = copies_wx[:,T+1], copies_x[:,T+1], copies_wy[:,T+1], copies_y[:,T+1]
nierrMP = glob_NI_err(gfun, wxMP, xMP, wyMP, yMP; deltax=deltax, deltay=deltay)
gvalMP = payoff(gfun, wxMP, xMP, wyMP, yMP)
open(logfile, "a") do f
    write(f, "Got last iterate (wxMP, xMP, wyMP, yMP) with NI error $(nierrMP)\n")
    write(f, "Payoff at (wxMP, xMP, wyMP, yMP): gvalMP=$(gvalMP)\n")
end
nierrMP, gvalMP

#### Compute and plot optimality metrics

In [ ]:
## plotting parameters
deltax = 1e-4 # discretization step when computing (approximation of) global NI error
deltay = 1e-4
evalNIevery    = Int(floor(T/500))
evallocNIevery = Int(floor(T/500))
evalVevery     = Int(floor(T/50))
plotiterevery  = Int(floor(T/50))

In [ ]:
### compute and plot (global) NI error of iterates
plt_NI, plt_NI_log, nierrs = plot_NI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evalNIevery;
    deltax=deltax, deltay=deltay, skip_avg=true)
save("$resultdir/nierrs__every$(evalNIevery).jld", "nierrs", nierrs)
plt_NI_log

In [ ]:
### compute and plot local NI error of iterates
plt_locNI, plt_locNI_log, locnierrs = plot_locNI_err(gfun, T, copies_wx, copies_x, copies_wy, copies_y, avg_wx, avg_x, avg_wy, avg_y,
    resultdir, logfile, evallocNIevery)
plt_locNI_log

In [ ]:
## plot first variations
plt_firstvar_x, plt_firstvar_y = plot_firstvars(gfun, wxMP, xMP, wyMP, yMP, gvalMP,
    resultdir)
plt_firstvar_x

In [ ]:
plt_firstvar_y